# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# defining humidity and locations for configuring maps
humidity = cities_df["Humidity"].astype(float)
max_humidity = humidity.max()
locations = cities_df[["Lat", "Lng"]]

In [15]:
# configuring gmaps with locations and humidity
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, 
                                 max_intensity=max_humidity, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the cities to fit weather conditions
ideal_city_df = cities_df.loc[(cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) 
                              & (cities_df["Wind Speed"] < 10) &(cities_df["Cloudiness"] == 0), :]
# Drop any rows will null values.
ideal_city_df = ideal_city_df.dropna(how='any')
ideal_city_df.reset_index(inplace=True)
del ideal_city_df['index']
ideal_city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_list = []

# use iterrows to iterate through ideal_city dataframe
for index, row in ideal_city_df.iterrows():
    lat = ideal_city_df.loc[index]['Lat']
    lng = ideal_city_df.loc[index]['Lng']

# Set parameters to search for hotels with 5000 meters.
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    # Hit the Google Places API for each city's coordinates.    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params)
    
    # extract results
    results = response.json()
    
    # Store the first Hotel result into the DataFrame.
    try:
        hotel_list.append(results['results'][0]['name'])
    except:
        hotel_list.append("")
        #print("------------")

# Add a "Hotel Name" column to the DataFrame.
ideal_city_df["Hotel Name"] = hotel_list
ideal_city_df = ideal_city_df.dropna(how='any')
ideal_city_df
    

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Capitólio
1,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,CIDREIRA
2,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Morondava
3,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Jagalbet
4,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Nador
5,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mogok
6,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand


In [22]:
# # Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)
# fig

NameError: name 'coordinates' is not defined

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_df.iterrows()]
locations = ideal_city_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))